# Partitioning

In this notebook you will partition data in the storage layout and see how to steere the number of generated files.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, year, round, rand
)

import os

In [ ]:
spark = (
    SparkSession
    .builder
    .appName('Partitioning I')
    .getOrCreate()
)

In [ ]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

questions_input_path = os.path.join(project_path, 'output/questions-transformed')

output_path_I = os.path.join(project_path, 'output/1/questions-partitioned')
output_path_II = os.path.join(project_path, 'output/2/questions-partitioned')


# Task I

* partition questions by year (derived from creation_date) and make one file per folder

In [ ]:
questionsDF = (
    spark
    .read
    .option('path', questions_input_path)
    .load()
    .withColumn('year', year('creation_date'))
)

#### Save the data:

Hint:
* repartition data by year to achive one file per folder

In [ ]:
(
    questionsDF
    .repartition('year')
    .write
    .mode('overwrite')
    .partitionBy('year')
    .option('path', output_path_I)
    .save()
)

# Task II

* partition questions by year (derived from creation_date) and make five files per folder

Hint:
* repartition data by year and random expression which generates random number from intrval [0, 4]

In [ ]:
(
    questionsDF
    .repartition('year', round(round(rand(12), 2) * 100) % 5)
    .write
    .mode('overwrite')
    .partitionBy('year')
    .option('path', output_path_II)
    .save()
)

In [ ]:
spark.stop()